In [10]:
%matplotlib inline
import pylab as pl
import numpy as np
from scipy import stats
import soundfile as sf
import os
import math
import sys

def readWavFile(name):
    frequencySample, signal = wavf.read(name)
    return frequencySample, signal

def preprocessSignal(signal):
    signal = makeMono(signal)
#     signal = removeSilence(signal)
    return signal

def makeMono(signal):
    finalSignal = []
    signal = signal.astype(np.float32)
    for s in signal:
        if isNumber(s):
            finalSignal.append(s)
        else:
            stereoMean = (s[0] + s[1]) / 2.0
            finalSignal.append(stereoMean)
            
    finalSignal = np.array(finalSignal)
        
    return finalSignal.astype(np.float32)

def isNumber(value):
    return not hasattr(value, "__len__")

def removeSilence(signal):
    finalSignal = []
    threshold = 400
    for s in signal:
        if abs(s) > threshold:
            finalSignal.append(s)
    
    return np.array(finalSignal)

def top_peaks(signal, freqSample, num = 10):
    freqs = pl.fftfreq(len(signal))
    freqs = freqs * freqSample
    for i in range(len(freqs)):
        if freqs[i]<50:
            signal[i]=0
        elif freqs[i]>250:
            signal[i]=0            
    peaks = []            
    for i in range(num):
        k = pl.argmax(signal)
        freq = freqs[k]
        peaks.append(freq)
        signal[k]=0        
    return peaks

def check_voice(wav_name):
    signal, freqSample = sf.read('train/' + wav_file)
    signal = preprocessSignal(signal)
    peaks = top_peaks(signal, freqSample, 100)
    #print(peaks)
    avr = np.mean(peaks)
#     print(stats.mode(peaks))
    if avr > 145:
        print(wav_name + ': ---K---: ' + str(avr))
    else:
        print(wav_name + ': ---M---: ' + str(avr))

if __name__ == '__main__':
#     frequencySample, signal = readWavFile(sys.argv[0]) # final version
#    freqSample, signal = readWavFile('train/005_M.wav') # test version
    for wav_file in os.listdir('train'):
        check_voice(wav_file)
#         try:
#             analyze_wav(wav_file)
#         except:
#             print(wav_file)
#             pass

    print('end')

aa
021_M.wav: ---M---: 132.241429688
011_M.wav: ---M---: 137.525671247
043_M.wav: ---K---: 152.730124081
091_M.wav: ---M---: 65.4576080853
052_M.wav: ---M---: 100.191727488
086_K.wav: ---M---: 82.652
037_K.wav: ---M---: 106.284363001
010_M.wav: ---M---: 139.723656716
031_K.wav: ---K---: 176.952691628
081_K.wav: ---M---: 84.4475658454
023_M.wav: ---M---: 0.0
029_K.wav: ---K---: 231.220972424
008_K.wav: ---K---: 175.616354711
084_M.wav: ---K---: 171.599435425
078_M.wav: ---M---: 70.5511221945
079_K.wav: ---M---: 77.4704968944
036_K.wav: ---M---: 97.9409657354
071_M.wav: ---M---: 116.290958046
058_M.wav: ---K---: 202.564047084
041_K.wav: ---M---: 85.2449138416
089_M.wav: ---K---: 201.867877493
049_M.wav: ---K---: 178.889316404
012_K.wav: ---K---: 170.574608281
048_K.wav: ---K---: 151.532337362
014_K.wav: ---M---: 0.0
085_K.wav: ---K---: 149.477850799
090_M.wav: ---M---: 0.0
063_M.wav: ---K---: 148.664682242
050_K.wav: ---K---: 156.513431768
072_K.wav: ---M---: 143.403219317
074_K.wav: ---

In [95]:
from aubio import source, pitch
import wave

def analyze_wav(wav_file):
    wav_file = 'train/' + wav_file
    win_s = 4096
    hop_s = 512
    file = wave.open(wav_file)
    samplerate = file.getframerate()

    s = source(wav_file, samplerate, hop_s)
    samplerate = s.samplerate
    pitch_o = pitch("yin", win_s, hop_s, samplerate)
    pitches = []
    while True:
        samples, read = s()
        pitch_sample = pitch_o(samples)[0]
        if pitch_sample > 0 and pitch_sample < 280:
            pitches += [pitch_sample]
        if read < hop_s: break
            
    avr = np.mean(pitches)
    q25 = np.percentile(pitches, 25)
    q75 = np.percentile(pitches, 75)
    iqr = q75 - q25
    if avr >= 140:
        gender = 'K'
    else:
        gender = 'M'
        
    print(wav_file + ' ~~~> ' + gender + ' ~~~> MEAN: ' + str(avr) + ' ~~~> IQR: ' + str(iqr) )
    return np.mean(pitches)


In [7]:
%matplotlib inline
import pylab as pl
import numpy as np
from scipy import stats
import scipy.io.wavfile as wavf
import os
import math
import sys
import soundfile as sf
from aubio import pitch

def makeMono(signal):
    finalSignal = []
    signal = signal.astype(np.float32)
    for s in signal:
        if isNumber(s):
            finalSignal.append(s)
        else:
            stereoMean = (s[0] + s[1]) / 2.0
            finalSignal.append(stereoMean)
            
    finalSignal = np.array(finalSignal)
        
    return finalSignal.astype(np.float32)

def isNumber(value):
    return not hasattr(value, "__len__")

def decisionTree(pitches):
    avr = np.mean(pitches)
    q25 = np.percentile(pitches, 25)
    q75 = np.percentile(pitches, 75)
    iqr = q75 - q25

    if avr >= 145:
        gender = 'K'
    elif iqr >= 135:
        gender = 'K'
    else:
        gender = 'M'

#     if avr >= 145:
#         gender = 'K'
#     else:
#         gender = 'M'
    
    return gender, avr, iqr

def analyze_wav(wav_file):
    wav_file = 'train/' + wav_file
    fileGender = wav_file[10]
    signal, samplerate = sf.read(wav_file)
    signal = makeMono(signal)
    
    win_s = 4096
    hop_s = 512
    pitch_o = pitch("yin", win_s, hop_s, samplerate)
    pitches = []
    i = 0
    size = 512
    n = len(signal)
    
    while size*(i+1) < n:
        samples = signal[(i*size):((i+1)*size)]
        pitch_sample  = pitch_o(samples)[0]
        if pitch_sample > 0 and pitch_sample < 280:
            pitches += [pitch_sample]
        i += 1
    
    gender, avr, iqr = decisionTree(pitches)
    print(wav_file + ' ~~~> ' + gender + ' ~~~> MEAN: ' + str(avr) + ' ~~~> IQR: ' + str(iqr) )
    if gender == fileGender:
        return 1
    print('NO')
    return 0


In [8]:
if __name__ == '__main__':
#     for wav_file in os.listdir('train'):
#         try:
#             analyze_wav(wav_file)
#         except:
#             print(wav_file)
#             pass
    good = 0

    for wav_file in os.listdir('train'):
        good += analyze_wav(wav_file)
        
    print(good)
    print('end')

train/021_M.wav ~~~> M ~~~> MEAN: 88.8512 ~~~> IQR: 62.5055074692
train/011_M.wav ~~~> M ~~~> MEAN: 76.0851 ~~~> IQR: 51.3999938965
train/043_M.wav ~~~> M ~~~> MEAN: 103.55 ~~~> IQR: 65.4138841629
train/091_M.wav ~~~> M ~~~> MEAN: 104.814 ~~~> IQR: 13.4621315002
train/052_M.wav ~~~> M ~~~> MEAN: 92.1828 ~~~> IQR: 23.218082428
train/086_K.wav ~~~> K ~~~> MEAN: 223.872 ~~~> IQR: 42.0978546143
train/037_K.wav ~~~> K ~~~> MEAN: 113.126 ~~~> IQR: 185.712995529
train/010_M.wav ~~~> M ~~~> MEAN: 142.54 ~~~> IQR: 14.3008728027
train/031_K.wav ~~~> K ~~~> MEAN: 160.181 ~~~> IQR: 26.5187835693
train/081_K.wav ~~~> K ~~~> MEAN: 133.43 ~~~> IQR: 193.471385956
train/023_M.wav ~~~> M ~~~> MEAN: 133.204 ~~~> IQR: 15.1418762207
train/029_K.wav ~~~> K ~~~> MEAN: 223.313 ~~~> IQR: 30.092414856
train/008_K.wav ~~~> K ~~~> MEAN: 222.57 ~~~> IQR: 51.0350036621
train/084_M.wav ~~~> M ~~~> MEAN: 119.561 ~~~> IQR: 124.20912838
train/078_M.wav ~~~> M ~~~> MEAN: 105.922 ~~~> IQR: 21.5302963257
train/079_K.wav ~